# Peak shaving and optimization

The power grid is designed to provide a reliable service. This often means that components are dimensioned to support the highest anticipated demand even if this occurs rarely. High load power peaks are undesirable, since they lead to oversized expensive infrastructure to avoid grid disruptions.

To persuade big consumers to reduce their peak power, their electricity bill includes, besides the regular energy consumption costs, charges for the maximum power measured during the billing period.
The consumers could reduce costs by adapting their consumption behavior, accommodating more flexible loads to low consumption times.
An alternative if this flexibility is not available, is to introduce a battery storage system to shift the loads and perform peak-shaving.
Overall the load profile will be flattened, while the consumption remains the same.

In this notebook we will:
* Explore peak-shaving with energy storage as a way to reduce power peak costs.
* Use optimization models to determine the peak-shaving strategy.
* Extend our optimization models to size our storage system.

In [1]:
import pandas as pd
import pyomo.environ as opt
import plotly.graph_objects as go

In [2]:
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Use case - dataset and base-scenario definition
For our example we take an industrial profile from the [*Standard Battery Application Profiles (SBAP)*](https://doi.org/10.1016/j.est.2019.101077). For simplicity, we will consider a single billing period of 1 month.

In [3]:
# load profile
profile = pd.read_csv("../data/industry_profile.csv", index_col=0, parse_dates=True)
profile.plot(template=template, labels={"value": "Power [kW]"})

<div class="alert alert-block alert-info">
<b>Task!</b> Calculate energy consumption, peak power and the resulting electricity bill.
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Hint! Pay attention when converting power to energy. </li>
</div>

In [4]:
# cost
electricity_cost =   0.12 # €/kWh
peak_power_cost  = 120.00 # €/kW

In [5]:
# SOL 0: Calculate energy consumption, peak power and the resulting electricity bill.
def baseline_analysis(profile, electricity_cost, peak_power_cost):
    demand = profile.sum() * 0.25
    peak = profile.max()
    cost = demand * electricity_cost + peak * peak_power_cost
    print(f"Consumption: {demand:10.2f} kWh")
    print(f"Peak power:  {peak:10.2f} kW")
    print(f"Total costs: {cost:10.2f} €")

In [6]:
print("--- without storage ---")
baseline_analysis(profile["load"], electricity_cost, peak_power_cost)

--- without storage ---
Consumption:  418545.10 kWh
Peak power:      868.74 kW
Total costs:  154474.72 €


## Optimization model - Operation

An energy storage system can reduce costs by performing peak shaving. But how exactly should it operate? 

We can define a target maximum power peak and charge or discharge the storage depending on if the load is currently above or below this threshold.
Defining a peak target could be nonetheless difficult, if the threshold is too high the storage system may not be used at its full potential, if it is too low, the storage system would not be able to fulfil it.

We can instead use mathematical optimization techniques to find the optimal storage charge/discharge schedule to best fulfill the task. The [*pyomo* library](https://www.pyomo.org/) allows us to easily formulate *linear programming* models with python.

<div class="alert alert-block alert-info">
<b>Task!</b> Formulate and optimize the model
<ol>
    <li> Formulate the optimization problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the time series results togther in a dataframe with the load profile. Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs. Evaluate the improvements against the scenario without storage system. </li>
    
</ol>
</div>

<div class="alert alert-block alert-warning">
SOL 1: Formulate the optimization problem in a markdown cell using \( \LaTeX \)

<b>Hint!</b> Read the <a href="https://pyomo.readthedocs.io/en/stable/">pyomo documentation</a>. </li>
</div>

$$
\begin{aligned}
&\min \sum_{T}(p^{grid,buy}_t \cdot \Delta t) \cdot k^{electricity} + p^{peak} \cdot k^{peak} \\
s.t.  &  \\
0 &\leq p^{ch}_t, \leq p^{N} \quad &\forall t \in T \\
0 &\leq p^{dch}_t, \leq p^{N} \quad &\forall t \in T \\
E^N \cdot SOC^{min} &\leq E^{bess}_t \leq E^N \cdot SOC^{max} \quad &\forall t \in T \\
E^{bess}_t &= E^{bess}_{t-1} + p^{ch}_t \Delta t \cdot \eta^{ch} - p^{dch}_t \Delta t \cdot (1/\eta^{dch}) \quad &\forall t \in T \\
E^{bess}_{t=0} &= E^{N} \cdot SOC^{init} \\ 
E^{bess}_{t=end} &\geq E^{N} \cdot SOC^{init} \\ 
p^{grid,buy}_t + p^{dch}_t &= p^{load}_t + p^{ch}_t \quad &\forall t \in T \\
p^{grid,buy}_t &\geq 0 \quad &\forall t \in T \\
p^{peak} &\geq 0  \\
p^{grid,buy}_t &\leq p^{peak} \quad &\forall t \in T \\
\end{aligned}
$$

**Variables**: Energy charge state at $t$ timestep $E^{bess}$, charge power at $t$ timestep $p^{ch}_t$, discharge power at $t$ timestep $p^{ch}_t$, grid power supply at $t$ timestep $p^{grid,buy}_t$, peak power $p^{peak}$.

**Parameters**: Storage system installed capacity $E^{N}$, storage system installed power $p^{N}$, electricity cost $k^{electricity}$, peak power cost $p^{peak}$, lower SOC limit $SOC^{min}$, upper SOC limit $SOC^{min}$, start SOC value $SOC^{init}$, charge efficiency $\eta^{ch}$, discharge efficiency $\eta^{dch}$, power demand at $t$ timestep $p^{load}_t$.

### Task 2:  Create a function <code>build_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>.


In [7]:
# Storage parameters
storage_params = {
    "capacity": 200.0,  # kWh
    "power": 200.0,      # kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.1,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

In [8]:
storage_params["effc"]

0.9

In [9]:
# task
def build_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # Optimization model
    model = opt.ConcreteModel()

    ## =====  Parameters  ===== ##
    n = len(load_profile)   # number of timesteps
    model.time = opt.RangeSet(0, n-1) # T
    dt = 0.25               # 15 min timesteps

    capacity  = storage_params["capacity"]
    max_power = storage_params["power"]
    
    soc_min, soc_max = storage_params["soc_bounds"]
    soc_start = storage_params["soc_start"]

    effc  = storage_params["effc"]
    effd  = storage_params["effd"]

    ## =====  Variables  ===== ##
    # BESS: power charge/discharge, energy content
    model.power_charge    = opt.Var(model.time, bounds=(0.0, max_power))
    model.power_discharge = opt.Var(model.time, bounds=(0.0, max_power))
    model.energy_bess     = opt.Var(model.time, bounds=(soc_min * capacity, soc_max * capacity))

    # Grid: power, peak-power limit (for peak-shaving)
    model.grid_power = opt.Var(model.time, within=opt.NonNegativeReals)
    model.peak_power = opt.Var(within=opt.NonNegativeReals)


    ## =====  Objective  ===== ##
    def objective_rule(m):
        return sum(m.grid_power) * dt * electricity_cost + m.peak_power * peak_power_cost
    
    model.obj = opt.Objective(sense=opt.minimize, rule = objective_rule)


    ## ===== Constraints ===== ##
    # SOC balance
    def energy_balance_rule(m, t):
        if t == m.time.first(): #
            return m.energy_bess[t] == soc_start * capacity + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
        return m.energy_bess[t] == m.energy_bess[t-1] + dt * (effc * m.power_charge[t] - (1/effd) * m.power_discharge[t])
    
    model.constraint_energy_balance = opt.Constraint(model.time, rule=energy_balance_rule)

    # SOC end conditions
    def energy_end_rule(m):
        return m.energy_bess[m.time.last()] >= soc_start * capacity
    
    model.constraint_energy_end = opt.Constraint(rule=energy_end_rule)

    # power balance
    def power_balance_rule(m, t):
        return m.grid_power[t] + m.power_discharge[t] - m.power_charge[t] == load_profile.load[t]

    model.constraint_power_balance = opt.Constraint(model.time, rule=power_balance_rule)

    # peak
    def peak_power_rule(m, t):
        return m.grid_power[t] <= m.peak_power
    
    model.constraint_peak_power = opt.Constraint(model.time, rule=peak_power_rule)

    return model

Herausforderung: Regeldefinition mit Pyomo:

Beispiel:


model.constraint_power_balance = opt.Constraint(model.time, rule=power_balance_rule)


Constriants über Zeitvektoren haben Länge der Zeitschritte der Opt Problems (model.time) sowie die eigentliche Regeldefinition für die Optimierungsvariablen


In [10]:
# task
model = build_model(
    storage_params=storage_params, 
    load_profile=profile, 
    electricity_cost=electricity_cost, 
    peak_power_cost=peak_power_cost
)

# Review des "Optimierungs-Problems"
 

In [11]:
model.pprint() 

1 RangeSet Declarations
    time : Dimen=1, Size=2976, Bounds=(0, 2975)
        Key  : Finite : Members
        None :   True : [0:2975]

5 Var Declarations
    energy_bess : Size=2976, Index=time
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
           0 :  20.0 :  None : 180.0 : False :  True :  Reals
           1 :  20.0 :  None : 180.0 : False :  True :  Reals
           2 :  20.0 :  None : 180.0 : False :  True :  Reals
           3 :  20.0 :  None : 180.0 : False :  True :  Reals
           4 :  20.0 :  None : 180.0 : False :  True :  Reals
           5 :  20.0 :  None : 180.0 : False :  True :  Reals
           6 :  20.0 :  None : 180.0 : False :  True :  Reals
           7 :  20.0 :  None : 180.0 : False :  True :  Reals
           8 :  20.0 :  None : 180.0 : False :  True :  Reals
           9 :  20.0 :  None : 180.0 : False :  True :  Reals
          10 :  20.0 :  None : 180.0 : False :  True :  Reals
          11 :  20.0 :  None : 180.0 : False :  True :  Rea

In [12]:
# task3: solve model
solver = opt.SolverFactory('glpk')  # glpk is an open source LP solver

In [13]:
status = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\hessehoh\AppData\Local\Temp\tmpp55yp7ze.glpk.raw --wglp
 C:\Users\hessehoh\AppData\Local\Temp\tmp83za70eo.glpk.glp --cpxlp C:\Users\hessehoh\AppData\Local\Temp\tmphq68nxy9.pyomo.lp
Reading problem data from 'C:\Users\hessehoh\AppData\Local\Temp\tmphq68nxy9.pyomo.lp'...
8930 rows, 11906 columns, 26785 non-zeros
65490 lines were read
Writing problem data to 'C:\Users\hessehoh\AppData\Local\Temp\tmp83za70eo.glpk.glp'...
62509 lines were written
GLPK Simplex Optimizer 5.0
8930 rows, 11906 columns, 26785 non-zeros
Preprocessing...
8928 rows, 11904 columns, 26782 non-zeros
Scaling...
 A: min|aij| =  2.250e-01  max|aij| =  1.000e+00  ratio =  4.444e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8928
      0: obj =   1.328040000e+05 inf =   1.674e+06 (2976)
   2975: obj =   2.370533116e+05 inf =   0.000e+00 (0) 29
*  3284: obj =   2.248128923e+05 inf = 

In [14]:
def recover_results(model, profile):
    df = profile.copy() # Solutions dataframe

    df["power"] = [opt.value(model.power_charge[t]) - opt.value(model.power_discharge[t]) for t in model.time]
    df["energy"]   = [opt.value(model.energy_bess[t]) for t in model.time]
    df["grid"]  = [opt.value(model.grid_power[t]) for t in model.time]

    return df

In [15]:
def analyze_results(df, electricity_cost, peak_power_cost):
    peak = df["load"].max()
    demand = df["load"].sum() * 0.25
    cost = demand * electricity_cost + peak * peak_power_cost
    
    opt_peak = df["grid"].max()
    opt_demand = df["grid"].sum() * 0.25
    opt_cost = opt_demand * electricity_cost + opt_peak * peak_power_cost
    
    delta_cost = (opt_cost - cost)/cost
    delta_peak = (opt_peak - peak)/peak
    delta_demand = (opt_demand - demand)/demand
    
    print(f"Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total costs: {opt_cost:10.2f} €   ({delta_cost:+7.2%})")


In [16]:
solution = recover_results(model, profile)
print("--- with storage ---")
analyze_results(solution, electricity_cost, peak_power_cost)

--- with storage ---
Consumption:  418597.69 kWh ( +0.01%)
Peak power:      766.74 kW  (-11.74%)
Total costs:  142240.62 €   ( -7.92%)


In [17]:
# task4: plot results 

def plot_storage_power(profile, model):
    powerc = [opt.value(model.power_charge[t]) for t in model.time]
    powerd = [- opt.value(model.power_discharge[t]) for t in model.time]
    
    fig = go.Figure()
    fig.update_layout(template=template)
    fig.add_trace(go.Scatter(x=profile.index, y=powerc, name="Power charge"))
    fig.add_trace(go.Scatter(x=profile.index, y=powerd, name="Power discharge"))
    fig.update_yaxes(title="Power [kW]")
    fig.update_traces(line_shape="hv")
    return fig

In [18]:
# PLOT: Load profile and grid power in the same plot.

In [19]:
solution[["load", "grid"]].plot(template=template, labels={"value": "Power [kW]"})

In [20]:
# PLOT:  Storage power. 


In [21]:
plot_storage_power(profile, model)

In [22]:
# PLOT:  Storage energy content or SOE (i.e. State of Energy).

solution["energy"].plot(template=template, labels={"value": "Energy [kWh]"})

## System dimension

We have succesfully optimized our storage operation to minimizes costs. This was done with a predefined system. But what if we find ourselfs on the planning stage, contemplating the idea of aquiring a storage system to perform peak shaving? The storage system comes with respective investment costs, so me might want to 
re-formulate the optimization problem with a further degree of freedom to find the best system size that minimizes the total costs.

Two storage system are considered, their specific costs and efficiency are described in the following table:

|                                   | Storage System 1  | Sotrage System 2  |
|:---------------------------------:|:-----------------:|:-----------------:|
| Specific capacity costs           | 50.00 €/kWh       | 15.00 €/kWh       |
| Specific power costs              | 20.00 €/kW        | 45.00 €/kW        |
| Efficiency charge ; discharge     | 95% ; 95%         | 90%  ; 90%        |

<div class="alert alert-block alert-info">
<b>Task!</b> Formulate and optimize the dimensioning model
<ol>
    <li> Formulate the optimal sizing problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_dimension_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the load profile and the time series results of both storage systems in a dataframe.</li>
    <li>  Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs for both systems. Which system would you choose? Give your arguments in a markdown cell. </li>
    
</ol>
</div>

TASK 1: LATEX


...
...
...


**Variables**: Storage system installed capacity $E^{N}$, storage system installed power $p^{N}$, energy charge state at $t$ timestep $E^{bess}$, charge power at $t$ timestep $p^{ch}_t$, discharge power at $t$ timestep $p^{ch}_t$, grid power supply at $t$ timestep $p^{grid,buy}_t$, peak power $p^{peak}$.

**Parameters**: Energy capacity specific investment costs $k^{inv,E}$, power specific investment costs $k^{inv,E}$, electricity cost $k^{electricity}$, peak power cost $p^{peak}$, lower SOC limit $SOC^{min}$, upper SOC limit $SOC^{min}$, start SOC value $SOC^{init}$, charge efficiency $\eta^{ch}$, discharge efficiency $\eta^{dch}$, power demand at $t$ timestep $p^{load}_t$.

In [23]:
# Storage parameters
storage1_params = {
    "capacity_cost": 50.00,  # €/kWh
    "power_cost": 20.00,     # €/kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.0,

    "effc": 0.95,     # charge efficiency
    "effd": 0.95,     # discharge efficiency
}

storage2_params = {
    "capacity_cost": 15.00,  # €/kWh
    "power_cost": 45.00,     # €/kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.0,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

In [24]:
# TASK 2
def build_dimension_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # Optimization model
    model = opt.ConcreteModel()

    ## =====  Parameters  ===== ##

    ...

    ## =====  Variables  ===== ##
    # BESS: power charge/discharge, energy content

    ...


    # BESS size

    ...

    # Grid: power, peak-power limit (for peak-shaving)

    ...


    ## =====  Objective  ===== ##

    ...

    ## ===== Constraints ===== ##
    # Power limits (charge & discharge)
    def power_charge_limit_rule(m, t):
        ...
    def power_discharge_limit_rule(m, t):
        ...
    # Energy content rule 
    def energy_max_rule(m, t):
        ...
    def energy_min_rule(m, t):
        ...

    # SOC balance
    def energy_balance_rule(m, t):
        ...
    
    # SOC end conditions
    def energy_end_rule(m):
        ...

    # power balance
    def power_balance_rule(m, t):
        ...

    # peak
    def peak_power_rule(m, t):
        ...
       

    return model

In [25]:
# task3: build models
model_dim1 = build_dimension_model(
    ...
)

TypeError: build_dimension_model() missing 3 required positional arguments: 'load_profile', 'electricity_cost', and 'peak_power_cost'

In [ ]:
model.pprint()

In [ ]:
#solve model 1
status = solver.solve(model_dim1, tee=True)

In [ ]:
model_dim2 = build_dimension_model(
    ...
)

In [ ]:
#solve model 2
status = solver.solve(model_dim2, tee=True)

In [ ]:
# task4: recover results
...

In [ ]:
# task4.1: analyze results
def analyze_dimension_results(df, model, storage_params, electricity_cost, peak_power_cost):
    # system dimension
    ...
    # operation
    ...
    
    
    print(f"Installed capacity: {capacity:7.2f} kWh")
    print(f"Installed power:    {power:7.2f} kW")
    print(f"System costs:       {system_cost:7.2f} €\n")
    
    print(f"Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total costs: {opt_cost:10.2f} €   ({delta_cost:+7.2%})")



In [ ]:
# task: analyze resuls
print("--- storage 1 ---")
analyze_dimension_results(res_dim1, model_dim1, storage1_params, electricity_cost, peak_power_cost)
print()
print("--- storage 2 ---")
analyze_dimension_results(res_dim2, model_dim2, storage2_params, electricity_cost, peak_power_cost)

In [ ]:
# task5: plot results

In [ ]:
# task5.1: plot load and grid power

In [ ]:
# task5.2: plot storage power

In [ ]:
# task5.2: plot Storage energy content (~SOC)